In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
from pprint import pprint
import time

from ctmmodels.const import *

In [2]:
_tmp = 2

phase_map = {
    (0,0,0): [(_tmp, LEFT_TURN, WESTBOUND), (_tmp, RIGHT_TURN, NORTHBOUND)], # WBL, NBR
    (0,0,1): [(_tmp, THROUGH_TURN, EASTBOUND), (_tmp, RIGHT_TURN, EASTBOUND)],
    (0,1,0): [(_tmp, LEFT_TURN, SOUTHBOUND), (_tmp, RIGHT_TURN, WESTBOUND)],
    (0,1,1): [(_tmp, THROUGH_TURN, SOUTHBOUND), (_tmp, RIGHT_TURN, SOUTHBOUND)],
    (1,0,0): [(_tmp, LEFT_TURN, EASTBOUND), (_tmp, RIGHT_TURN, SOUTHBOUND)],
    (1,0,1): [(_tmp, THROUGH_TURN, WESTBOUND), (_tmp, RIGHT_TURN, WESTBOUND)],
    (1,1,0): [(_tmp, LEFT_TURN, NORTHBOUND), (_tmp, RIGHT_TURN, EASTBOUND)],
    (1,1,1): [(_tmp, THROUGH_TURN, NORTHBOUND), (_tmp, RIGHT_TURN, NORTHBOUND)],
}

In [5]:
# Source cells: (0,approach_id)
set_C_O = [(CELL_SOURCE,0,i)
    for i in range(APPROACHES)]

# Sink cells: (0,approach_id)
set_C_S = [(CELL_SINK,0,i)
    for i in range(APPROACHES)]

# Movement cells: (movement_id, apporach_id)
set_C_I = [(CELL_MOVEMENT,i,j)
    for i in range(MOVEMENT_CELLS)
    for j in range(APPROACHES)]

# Normal cells: (cell_id, approach_id)
set_C_N = [(CELL_NORMAL,i,j)
    for i in range(APPROACH_CELLS)
    for j in range(APPROACHES)]

# Set of all cells: (cell_type, x, y)
set_C = set_C_O + set_C_S + set_C_I + set_C_N

## Computing delay equity

In [3]:
dfx = pd.read_pickle('x.pkl')
dfy = pd.read_pickle('y.pkl')

In [4]:
dfy.head()

,timestep,cell_from,cell_to,flow
0,54,"(2, 0, 0)","(1, 0, 1)",0.0
1,47,"(2, 0, 3)","(1, 0, 0)",0.0
2,13,"(0, 0, 3)","(3, 0, 3)",1.5
3,53,"(3, 1, 1)","(3, 2, 1)",2.0
4,22,"(2, 2, 0)","(1, 0, 3)",0.0


In [55]:
D_term = self.model.sum(
            self.model.sum(
                self.x_vars[(i,t)] - self.model.sum(
                    self.y_vars[(i,j,t)]
                    for j in self.S[i])
                for i in self.set_C if i not in self.set_C_S)
            for t in self.set_T)

In [56]:
dfg_conv.is_green = pd.to_numeric(dfg_conv.is_green, 'is_green', downcast='integer')
dfg_conv.head()

,timestep,cell,is_green
479,13,"(2, 1, 1)",0
478,13,"(2, 2, 1)",0
477,25,"(2, 0, 2)",0
476,25,"(2, 2, 3)",0
475,6,"(2, 0, 1)",0


In [73]:
dfg_matrix = dfg_conv.pivot_table(columns='cell', index='timestep', values='is_green', aggfunc=np.sum)

In [74]:
dfg_matrix

cell,"(2, 0, 0)","(2, 0, 1)","(2, 0, 2)","(2, 0, 3)","(2, 1, 0)","(2, 1, 1)","(2, 1, 2)","(2, 1, 3)","(2, 2, 0)","(2, 2, 1)","(2, 2, 2)","(2, 2, 3)"
timestep,,,,,,,,,,,,
0,1,0,0,0,0,0,1,0,0,1,1,0
1,1,0,0,0,0,0,1,0,0,1,1,0
2,0,0,0,0,0,1,0,1,0,1,0,1
3,0,0,0,0,0,1,0,1,0,1,0,1
4,0,0,0,0,0,1,0,1,0,1,0,1
5,1,0,1,0,0,0,0,0,0,1,0,1
6,1,0,1,0,0,0,0,0,0,1,0,1
7,1,0,1,0,0,0,0,0,0,1,0,1
8,0,1,0,1,0,0,0,0,1,0,1,0


In [80]:
dfg_matrix.replace(0,2)

cell,"(2, 0, 0)","(2, 0, 1)","(2, 0, 2)","(2, 0, 3)","(2, 1, 0)","(2, 1, 1)","(2, 1, 2)","(2, 1, 3)","(2, 2, 0)","(2, 2, 1)","(2, 2, 2)","(2, 2, 3)"
timestep,,,,,,,,,,,,
0,1,2,2,2,2,2,1,2,2,1,1,2
1,1,2,2,2,2,2,1,2,2,1,1,2
2,2,2,2,2,2,1,2,1,2,1,2,1
3,2,2,2,2,2,1,2,1,2,1,2,1
4,2,2,2,2,2,1,2,1,2,1,2,1
5,1,2,1,2,2,2,2,2,2,1,2,1
6,1,2,1,2,2,2,2,2,2,1,2,1
7,1,2,1,2,2,2,2,2,2,1,2,1
8,2,1,2,1,2,2,2,2,1,2,1,2


In [50]:
timerange, phases = dfg_matrix.shape

In [53]:
# dfg_matrix.iloc[0,:] = [2]*phases

for t in range(1,timerange):
    for p in range(phases):
        if dfg_matrix.iloc[t,p] == 1 and dfg_matrix.iloc[t-1,p] == 0:
            dfg_matrix.iloc[t,:] = [2]*phases

In [54]:
dfg_matrix

cell,"(0, 0, 0)","(0, 0, 1)","(0, 1, 0)","(0, 1, 1)","(1, 0, 0)","(1, 0, 1)","(1, 1, 0)","(1, 1, 1)"
timestep,,,,,,,,
0,2,2,2,2,2,2,2,2
1,0,0,1,0,0,0,0,1
2,2,2,2,2,2,2,2,2
3,0,1,0,0,0,1,0,0
4,0,1,0,0,0,1,0,0
5,2,2,2,2,2,2,2,2
6,0,0,1,0,0,0,1,0
7,0,0,1,0,0,0,1,0
8,2,2,2,2,2,2,2,2


In [37]:
dfg_matrix[(1,0,1)].astype('int16').values

array([0, 0, 2, 1, 1, 2, 0, 0, 2, 0, 0, 0, 2, 0, 0, 2, 1, 1, 2, 0, 0, 0,
       2, 0, 0, 2, 0, 0, 2, 1], dtype=int16)